In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [ ]:
# Create a DataFrame with the scaled data
df_scaled_market_data = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_scaled_market_data["coin_id"] = df_market_data.index

# Set the coinid column as index
df_scaled_market_data = df_scaled_market_data.set_index("coin_id")

# Display sample data
df_scaled_market_data.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = range(1,12)


In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=i)
# 2. Fit the model to the data using `df_market_data_scaled`
    model.fit(df_scaled_market_data)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(model.inertia_)
    
inertia

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
inertia_dict = {'k':k, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
inertia_df = pd.DataFrame(inertia_dict)
inertia_df = inertia_df.set_index('k')
inertia_df

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

original_data = inertia_df['inertia'].hvplot.line(
    x ='k', 
    y ='inertia', 
    title  = 'Inertia v Number of Clusters (k)',
    xlabel = 'Number of Clusters (k)',
    ylabel = 'Inertia',
    width = 500,
    xticks = 'k'
    )

original_data

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value of k is k=4


---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_scaled_market_data)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters = model.predict(df_scaled_market_data)

# Print the resulting array of cluster values.
clusters

In [ ]:
# Create a copy of the DataFrame
df_scaled_market_data2 = df_scaled_market_data.copy()


In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_scaled_market_data2['cluster_original'] = clusters

# Display sample data
df_scaled_market_data2.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

plot_cluster_data = df_scaled_market_data2.hvplot.scatter(
    x     = "price_change_percentage_24h", 
    y     = "price_change_percentage_7d", 
    by    = 'cluster_original',
    title = "Original Data. K-Mean Clusters with k=4.",
    width = 500,
    hover_cols = 'coin_id'
)

plot_cluster_data


---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components = 3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_market_data = pca.fit_transform(df_scaled_market_data)

# View the first five rows of the DataFrame. 
pca_market_data[0:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
# Retrieve the explained variance to determine how much information can be attributed to each principal component.
pca.explained_variance_ratio_

In [ ]:
#****** This cell added to calculate the total explained variance of the three components *******
pca.explained_variance_ratio_.sum()

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance is 0.895 which gives a value of 90% 

In [ ]:
# Create a new DataFrame with the PCA data.
df_pca_market_data = pd.DataFrame(pca_market_data, columns=['PC1','PC2','PC3'])


# Copy the crypto names from the original data
df_pca_market_data['coin_id'] = df_scaled_market_data.index

# Set the coinid column as index
df_pca_market_data = df_pca_market_data.set_index('coin_id')

# Display sample data
df_pca_market_data.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))

In [ ]:
# Create an empy list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k

for i in k:
    # Inside the loop:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    model_pca = KMeans(n_clusters = i)
    # 2. Fit the model to the data using `df_market_data_pca`
    model_pca.fit(df_pca_market_data)
    # 3. Append the model.inertia_ to the inertia list
    pca_inertia.append(model_pca.inertia_)

pca_inertia

In [ ]:
# # Create a dictionary with the data to plot the Elbow curve
# # Create a DataFrame with the data to plot the Elbow curve

k = list(range(1,12))
# Create a dictionary with the data to plot the Elbow curve
pca_inertia_dict = {'k':k, 'pca_inertia':pca_inertia}

# Create a DataFrame with the data to plot the Elbow curve
pca_inertia_df = pd.DataFrame(pca_inertia_dict)

pca_inertia_df = pca_inertia_df.set_index('k')
pca_inertia_df


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

pca_data = pca_inertia_df['pca_inertia'].hvplot.line(
    x="k", 
    y="pca_inertia",
    title  = 'Inertia v Number of Clusters (k)',
    xlabel = 'Number of Clusters (k)',
    ylabel = 'Inertia',
    width = 500,
    color = 'Red'
    )

pca_data

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for 'k' when using the PCA processed data is k = 4 with an inertia of 49.67.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The best value in the original data was also considered to be k = 4 but it should be noted the inertia at this value was 79.02.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
k = 4
model_pca = KMeans(n_clusters=k, random_state=1)

In [ ]:
# Fit the K-Means model using the PCA data
model_pca.fit(df_pca_market_data)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
cluster = model_pca.predict(df_pca_market_data)

# Print the resulting array of cluster values.
cluster[0:5]

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.


# Add a new column to the DataFrame with the pca predicted clusters
df_scaled_market_data3 = df_scaled_market_data2.copy()

df_scaled_market_data3['pca_pr-clusters'] = cluster
#df_scaled_market_data3.head()

plot_cluster2_data = df_scaled_market_data3.hvplot.scatter(
    x     = "price_change_percentage_24h", 
    y     = "price_change_percentage_7d", 
    by    = 'pca_pr-clusters',
    title = "PCA Processed Data. K-Mean Clusters with k=4.",
    width = 500,
    hover_cols = 'coin_id'
)

plot_cluster2_data



### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
original_data + pca_data

In [ ]:
# Composite plot to contrast the Elbow curves

# additional plot for clarity
original_data * pca_data


In [ ]:
# Composite plot to contrast the clusters

plot_cluster_data + plot_cluster2_data


#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The cluster analysis results show that the same datapoints are assigned to the same group of clusters and therefore the two processes have achieved the same result. However the PCA processed data has achieved the result in a more efficient manner. There is no negative impact by using the PCA process to analyse fewer features.